In [ ]:
from os import path

# Third-party
import astropy.coordinates as coord
from astropy.table import Table, vstack
from astropy.io import fits, ascii
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from pyia import GaiaData

import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.dynamics import mockstream
from gala.units import galactic

In [ ]:
g = GaiaData(Table.read('gd1_tbl.fits'))
gd1_gal = coord.Galactic(l=g[g.parallax>0].skycoord.galactic.l,
                         b=g[g.parallax>0].skycoord.galactic.b)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.plot(gd1_gal.l.degree, gd1_gal.b.degree, marker='.', linestyle='none')
ax.set_xlim(300, 100)
ax.set_ylim(0, 80)

### Distance from Sergey's paper:

In [ ]:
c = coord.SkyCoord(ra=g.ra, dec=g.dec,
                   pm_ra_cosdec=g.pmra, pm_dec=g.pmdec)
gd1_c = c.transform_to(gc.GD1)

In [ ]:
def gd1_dist(phi1):
    # 0, 10
    # -60, 7
    m = (10-7) / (60)
    return (m*phi1.wrap_at(180*u.deg).value + 10) * u.kpc

In [ ]:
mask = (np.abs(gd1_c.phi2) < 0.5*u.deg) & (gd1_c.phi1 > 300*u.deg)  & (gd1_c.phi1 < 310*u.deg)

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

ax = axes[0]
ax.plot(g.pmra[mask], g.pmdec[mask], 
        marker=',', linestyle='none', alpha=0.7)

ax.set_xlim(-30, 30.)
ax.set_ylim(-30, 30)
ax.set_xlabel(r'$\mu_\alpha$')
ax.set_ylabel(r'$\mu_\delta$')

ax = axes[1]
ax.plot(gd1_c.pm_phi1_cosphi2.to(u.mas/u.yr)[mask], 
        gd1_c.pm_phi2.to(u.mas/u.yr)[mask], 
        marker='.', linestyle='none', alpha=0.7)

ax.set_xlim(-20, -5.)
ax.set_ylim(-10, 5)
ax.set_xlabel(r'$\mu_{\phi,1}$')
ax.set_ylabel(r'$\mu_{\phi,2}$')

fig.set_facecolor('w')
fig.tight_layout()

In [ ]:
(np.median(gd1_c.pm_phi1_cosphi2.to(u.mas/u.yr)[mask]), 
 np.median(gd1_c.pm_phi2.to(u.mas/u.yr)[mask]))

In [ ]:
w0_gd1 = gc.GD1(phi1=300*u.deg, phi2=0*u.deg, 
                distance=gd1_dist(coord.Angle(300*u.deg)),
                pm_phi1_cosphi2=np.median(gd1_c.pm_phi1_cosphi2.to(u.mas/u.yr)[mask]),
                pm_phi2=np.median(gd1_c.pm_phi2.to(u.mas/u.yr)[mask]),
                radial_velocity=120*u.km/u.s)

w0 = gd.PhaseSpacePosition(w0_gd1.transform_to(coord.Galactocentric).cartesian)

In [ ]:
mw = gp.Hamiltonian(gp.MilkyWayPotential())

In [ ]:
orbit = mw.integrate_orbit(w0, dt=-1*u.Myr, n_steps=4000)

In [ ]:
stream = mockstream.fardal_stream(mw, orbit[::-1], 1e5*u.Msun, release_every=1)

In [ ]:
_ = stream.plot()

In [ ]:
stream_gd1 = stream.to_coord_frame(gc.GD1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
ax.plot(stream_gd1.phi1, stream_gd1.phi2, marker='.', alpha=0.5, linestyle='none')
ax.set_ylim(-5, 5)
ax.set_xlim(300, 360)

In [ ]:
chunk_mask = np.logical_not((stream_gd1.phi1 > 338*u.deg) & (stream_gd1.phi1 < 342*u.deg))
chunk_mask &= (stream_gd1.phi1 > 300*u.deg) & (stream_gd1.phi1 < 352*u.deg)
chunk_mask &= np.abs(stream_gd1.phi2) < 2*u.deg

fig, ax = plt.subplots(1, 1, figsize=(12, 4))
ax.plot(stream_gd1.phi1[chunk_mask], stream_gd1.phi2[chunk_mask], 
        marker='.', alpha=0.5, linestyle='none')
ax.set_ylim(-5, 5)
ax.set_xlim(300, 360)

In [ ]:
def reflex(c):
    c_oph_hack = gc.GD1(phi1=c.phi1, phi2=c.phi2,
                        distance=gd1_dist(c.phi1),
                        pm_phi1_cosphi2=c.pm_phi1_cosphi2,
                        pm_phi2=c.pm_phi2,
                        radial_velocity=[0]*len(c)*u.km/u.s)
    
    # Correct for reflex motion
    v_sun = coord.Galactocentric.galcen_v_sun
    observed = c_oph_hack.transform_to(coord.Galactic)
    rep = observed.cartesian.without_differentials()
    rep = rep.with_differentials(observed.cartesian.differentials['s'] + v_sun)
    return coord.Galactic(rep).transform_to(gc.GD1)

In [ ]:
stream_gd1_reflex = reflex(stream_gd1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.plot(stream_gd1_reflex.pm_phi1_cosphi2[chunk_mask].to(u.mas/u.yr), 
        stream_gd1_reflex.pm_phi2[chunk_mask].to(u.mas/u.yr), 
        marker='.', alpha=0.1, linestyle='none')
ax.set_xlim(-12, -2)
ax.set_ylim(-4, 4)

In [ ]:
plt.hist(g.pmra_error.value, bins='auto');
plt.hist(g.pmdec_error.value, bins='auto');
np.median(g.pmra_error.value), np.median(g.pmdec_error.value)

In [ ]:
obs_pm_phi1 = np.random.normal(stream_gd1_reflex.pm_phi1_cosphi2[chunk_mask].to(u.mas/u.yr).value, 0.2)
obs_pm_phi2 = np.random.normal(stream_gd1_reflex.pm_phi2[chunk_mask].to(u.mas/u.yr).value, 0.2)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.plot(obs_pm_phi1, obs_pm_phi2,
        marker='.', alpha=0.25, linestyle='none')
ax.set_xlim(-12, -2)
ax.set_ylim(-4, 4)